# IMPORTS

In [19]:
import os
import tempfile

import pandas as pd
import numpy as np
from configparser import ConfigParser

from tf_keras.src.layers import GlobalMaxPooling2D, Dense, Dropout

os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_keras import losses,optimizers, models, layers, Input
from tf_keras.applications import ResNet50
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from functools import partial
from utils import load_dataset, reformat_image, entropy_prune_model,time_benchmark, normalize_img, data_augmentation, unstructured_prune_model, create_model_checkpoint
from utils.Objects import PrunedModel, EntropyPruningSurgeon, L1NormPruning
from utils.Plots import plot_unstructured_accuracy_sparcity

# CONSTANTS



In [17]:
config = ConfigParser()
config.read("config.ini")

BATCH_SIZE = int(config["BATCHES"]["BATCH_SIZE"])
IMG_SIZE = (int(config["IMAGES"]["IMG_SIZE_RESNET"]), int(config["IMAGES"]["IMG_SIZE_RESNET"]))

MODEL_DIRECTORY = config["MODEL"]["MODEL_DIRECTORY"]
WEIGHTS_DIRECTORY = config["MODEL"]["WEIGHTS_DIRECTORY"]
METRICS_DIRECTORY = config["MODEL"]["METRICS_DIRECTORY"]

UNSTRUCTURED_FLOWERS_WEIGHTS = os.path.join(WEIGHTS_DIRECTORY, config["MODEL"]["UNSTRUCTURED_FLOWERS_WEIGHTS"])
FLOWERS_MODEL = os.path.join(MODEL_DIRECTORY, config["MODEL"]["FLOWERS_MODEL"])
UNSTRUCTURED_METRICS = os.path.join(METRICS_DIRECTORY, config["MODEL"]["UNSTRUCTURED_METRICS"])
IMG_SIZE, BATCH_SIZE

((300, 300), 32)

In [4]:
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset("tf_flowers", 70)


INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1.
INFO:absl:Constructing tf.data.Dataset tf_flowers for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1


In [5]:
train_examples

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [6]:
len(train_examples), len(validation_examples)

(2569, 1101)

In [13]:
def reformat_image(image, label, image_size=IMG_SIZE):
    image = tf.image.resize(image, image_size)
    return image, label

In [8]:
# Data augmentation
train_examples = train_examples.map(data_augmentation)
data_training = train_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
data_validation = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
data_training, data_validation

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

# AUTOTUNE

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
dataset_training = data_training.cache().prefetch(buffer_size=AUTOTUNE)
dataset_validation = data_validation.cache().prefetch(buffer_size=AUTOTUNE)
dataset_training, dataset_validation

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

# Base Model

In [10]:
base_model = ResNet50(input_shape=IMG_SIZE + (3,), include_top=False, weights="imagenet")

# Freeze the base model

In [11]:
base_model.trainable = False
max_pool_layer = GlobalMaxPooling2D()
prediction_layer = Dense(num_classes, activation="softmax")

# Build the Model

In [12]:
inputs = Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = max_pool_layer(x)
x = Dropout(0.5)(x)
outputs = prediction_layer(x)
model = models.Model(inputs, outputs, name="flowers_resnet50.keras")
model.summary()

Model: "flowers_resnet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 10, 10, 2048)      23587712  
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 23597957 (90.02 MB)
Trainable params: 10245 (40.02 KB)
Non-trainable params: 23587712 (89.98 MB)
_____

In [17]:
base_learning_rate = 0.001
model.compile(loss=losses.SparseCategoricalCrossentropy(),
                optimizer=optimizers.Adam(learning_rate=base_learning_rate),
                metrics=["accuracy"])


In [19]:
initial_epochs = 10
model_history = model.fit(dataset_training,
                              validation_data=dataset_training,
                              batch_size=BATCH_SIZE,
                              epochs=initial_epochs,
                              callbacks=[create_model_checkpoint(model_name=model.name)])

Epoch 1/10


Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [28]:

base_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 15

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:-fine_tune_at]:
  layer.trainable =  False

In [29]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
                optimizer=optimizers.Adam(learning_rate=base_learning_rate/10),
                metrics=["accuracy"])

In [30]:
fine_tune_epochs = 15
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(dataset_training,
                         epochs=total_epochs,
                         initial_epoch=model_history.epoch[-1] + 1,
                         validation_data=dataset_training,
                         callbacks=[create_model_checkpoint(model_name=model.name)])

NameError: name 'model_history' is not defined

In [32]:
model_1 = models.load_model("model_experiments/flowers_resnet50.keras")
model_1.evaluate(dataset_training)

OSError: No file or directory found at model_experiments/flowers_resnet50

In [14]:
val_data = validation_examples
val_data = val_data.map(reformat_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(normalize_img)

# Evaluate the model time

In [15]:
init_time, avg_time, std = time_benchmark(model_1, class_names, val_data)

NameError: name 'model_1' is not defined

# Pruning the model unstructured

In [18]:
k_sparsities = [0.25, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.97, 0.99]

metric_list = []

for k in k_sparsities:
    model_1 = models.load_model("model_experiments/flowers_resnet50.keras")
    pruned_model: PrunedModel = unstructured_prune_model(model_1, k, 20,data_training, data_validation, BATCH_SIZE)
    metrics = pruned_model.metrics
    print(f"{metrics.sparsity=}, \t {metrics.val_loss=}, \t {metrics.val_accuracy=}")
    metric_list.append(metrics.to_df())

df = pd.concat(metric_list)
df.to_csv(UNSTRUCTURED_METRICS)
df

OSError: No file or directory found at model_experiments/flowers_resnet50

In [ ]:
data_2 = pd.read_csv(UNSTRUCTURED_METRICS)
plot_unstructured_accuracy_sparcity(data_2)

# Final Model Pruning

In [20]:
model_2 = models.load_model("model_experiments/flowers_resnet50.keras")

final_sparsity = 0.7

pruned_model_final = unstructured_prune_model(model_1, final_sparsity, 20,data_training, data_validation, BATCH_SIZE)

final_pruned_model = pruned_model_final.model

final_pruned_model.save(FLOWERS_MODEL + "_pruned_" + str(final_sparsity) + ".keras")
final_pruned_model.summary()

OSError: No file or directory found at model_experiments/flowers_resnet50.keras

# Pruning the model with entropy

In [ ]:
test_threshold = 0.028
final_model = models.load_model("model_experiments/flowers_resnet50.kera")
entropy = EntropyPruningSurgeon(model=final_model, threshold=test_threshold)
pruned_model = entropy.run()
pruned_model.summary()

# Pruning the model with L1- Norm

In [ ]:
k_l1_norm = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.4, 0.5]
metric_list = []

for k in k_l1_norm:
    model_1 = models.load_model("model_experiments/custom_vgg16.keras")
    pruned_model: PrunedModel = l1_norm_prune_model(model_1,20,k,train_batches,validation_batches,BATCH_SIZE)
    metrics = pruned_model.metrics
    val_loss = metrics["val_loss"]
    val_accuracy = metrics["val_accuracy"]
    print(f"L1_norm_threshold={k}, \t {val_loss=}, \t {val_accuracy=}")
    metric_list.append(metrics)

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)
df.to_csv(L1_NORM_METRICS)
df

# Apply a compression algorithm to see the benefits of pruning

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(final_pruned_model)
model_for_export.save_weights(UNSTRUCTURED_FLOWERS_WEIGHTS)
model_for_export.summary()

# Need to convert the model to a TFLite model

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

NameError: name 'model_for_export' is not defined

# Apply the Quantization algorithm to the model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)